In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [3]:
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=74a08bbdeddc4ae215c8646986c4b86473e2aaa5aa5254b0661dcc92b164b4ca
  Stored in directory: /root/.cache/pip/wheels/1e/5c/47/759b32beb377aee150eb4c1f1b2b7635032cc19ea09ce076fd
Successfully built pandas_ta


In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/')

In [4]:
import pandas_datareader.data as pdr

def get_stock_data(code):
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [5]:
import plotly.graph_objs as go
#import talib as ta
import pandas_ta as ta
import datetime as dt
import pandas as pd
import numpy as np

name = '積水ハウス'
code = 1928
df = get_stock_data(code)
close = df["Close"]

# 5日、25日移動平均の算出
#ma5, ma25 = ta.SMA(close, timeperiod=5), ta.SMA(close, timeperiod=25)
ma5, ma25 = ta.sma(close ,length=5), ta.sma(close ,length=25)
df["ma5"], df["ma25"] = ma5, ma25

ma5, ma25 = df["ma5"], df["ma25"]
cross  = ma5 > ma25

cross_shift = cross.shift(1)
temp_gc = (cross != cross_shift) & (cross == True)
temp_dc  = (cross != cross_shift) & (cross == False)

# ゴールデンクロス発生日であればMA5の値、それ以外はNaN
gc = [m if g == True else np.nan for g, m in zip(temp_gc, ma5)]

# デッドクロス発生日であればMA25の値、それ以外はNan
dc = [m if d == True else np.nan for d, m in zip(temp_dc, ma25)]

# データフレームのカラムとして保存
df["gc"], df["dc"] = gc, dc

# ボリンジャーバンドの算出
#upper2, _, lower2 = ta.BBANDS(close, timeperiod=25, nbdevup=2, nbdevdn=2, matype=0)
#df['upper2'], df['lower2'] = upper2, lower2

bb2 = ta.bbands(close, length=25, std=2.0, mamode="sma")
df["upper2"], df["lower2"] = bb2["BBU_25_2.0"], bb2["BBL_25_2.0"]

# MACD、シグナル、ヒストグラムを算出
#macd, macdsignal, hist = ta.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
#df["macd"] = macd
#df["macd_signal"] = macdsignal
#df["hist"] = hist

mdf = ta.macd(close, fast=12, slow=26, signal=9)
df['macd'] = mdf['MACD_12_26_9']
df['macd_signal'] = mdf['MACDs_12_26_9']
df['hist'] = mdf['MACDh_12_26_9']

# RSI
#rsi14 = ta.RSI(close, timeperiod=14)
#rsi28 = ta.RSI(close, timeperiod=28)

rsi14 = ta.rsi(close, length=14)
rsi28 = ta.rsi(close, length=28)
df["rsi14"], df["rsi28"] = rsi14, rsi28

df["70"],  df["30"] = [70 for _ in close], [30 for _ in close]

# ストキャスティクス
#slowK, slowD = ta.STOCH(df["High"], df["Low"], df["Close"],
#                                fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
#df["slowK"], df["slowD"] = slowK, slowD

sdf = ta.stoch(df["High"], df["Low"], close, k=5, d=3, smooth_k=3, mamode="sma")
df['slowK'], df['slowD'] = sdf['STOCHk_5_3_3'], sdf['STOCHd_5_3_3']


df["80"],  df["20"] = [80 for _ in close], [20 for _ in close]

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

layout = {
            "height": 1000,
            "title"  : { "text": "{} {}".format(code, name), "x":0.5 }, 
            "xaxis" : { "rangeslider": { "visible": False } },
            "yaxis1": { "domain": [.46, 1.0], "title": "価格（円）", "side": "left", "tickformat": "," },
            "yaxis2": { "domain": [.40, .46] }, # x
            "yaxis3": { "domain": [.30, .395], "title": "MACD", "side": "right"}, # MACD
            "yaxis4": { "domain": [.20, .295], "title": "RSI", "side": "right"}, # RSI
            "yaxis5": { "domain": [.10, .195], "title": "STC", "side": "right"}, # ストキャスティクス
            "yaxis6": { "domain": [.00, .095], "title": "Volume", "side": "right"}, # 出来高
            "plot_bgcolor":"light blue"
          }

data =  [
            # ローソク足
            go.Candlestick(yaxis="y1", x=rdf.index, open=rdf["Open"], high=rdf["High"], low=rdf["Low"], close=rdf["Close"],   
                            increasing_line_color="red", decreasing_line_color="gray"),
         
            # 5日移動平均線
            go.Scatter(yaxis="y1", x=rdf.index, y=rdf["ma5"], name="MA5", line={ "color": "royalblue", "width": 1.2 } ),
         
            # 25日移動平均線
            go.Scatter(yaxis="y1", x=rdf.index, y=rdf["ma25"], name="MA25", line={ "color": "lightseagreen", "width": 1.2 } ),
         
            # ゴールデンクロス
            go.Scatter(yaxis="y1", x=rdf.index, y=rdf["gc"], name="Golden Cross", mode="markers", marker={ "size": 15, "color": "purple" }, opacity = 0.6),
         
            # デッドクロス
            go.Scatter(yaxis="y1", x=rdf.index, y=rdf["dc"], name="Dead Cross", mode="markers", marker={ "size": 15, "color": "black", "symbol": "x" },  opacity = 0.8),
            
            # ボリンジャーバンド
            go.Scatter(yaxis="y1", x=rdf.index ,y=rdf["upper2"], name= "", line={ "color": "lavender", "width": 0 }),
            go.Scatter(yaxis="y1", x=rdf.index ,y=rdf["lower2"], name= "BB", line={ "color": "lavender", "width": 0}, 
                    fill="tonexty", fillcolor="rgba(170,170,170,.2)"),
         
            # MACD
            go.Scatter(yaxis="y3" ,x=rdf.index ,y=rdf["macd"], name= "macd", line={ "color": "magenta", "width": 1 } ),
            go.Scatter(yaxis="y3" ,x=rdf.index ,y=rdf["macd_signal"], name= "signal", line={ "color": "green", "width": 1 } ),
            go.Bar(yaxis="y3" ,x=rdf.index, y=rdf["hist"], name= "histgram", marker={ "color": "slategray" } ) ,

            # RSI
            go.Scatter(yaxis="y4" ,x=rdf.index ,y=rdf["rsi14"], name= "RSI14",line={ "color": "magenta", "width": 1 } ),
            go.Scatter(yaxis="y4" ,x=rdf.index ,y=rdf["rsi28"], name= "RSI28",line={ "color": "green", "width": 1 } ),
            go.Scatter(yaxis="y4" ,x=rdf.index ,y=rdf["30"], name= "30",line={ "color": "black", "width": 0.5 } ),
            go.Scatter(yaxis="y4" ,x=rdf.index ,y=rdf["70"], name= "70",line={ "color": "black", "width": 0.5 } ),

            # ストキャスティクス
            go.Scatter(yaxis="y5" ,x=rdf.index ,y=rdf["slowK"], name= "slowK",line={ "color": "magenta", "width": 1 } ),
            go.Scatter(yaxis="y5" ,x=rdf.index ,y=rdf["slowD"], name= "slowD",line={ "color": "green", "width": 1 } ),
            go.Scatter(yaxis="y5" ,x=rdf.index ,y=rdf["20"], name= "20",line={ "color": "black", "width": 0.5 } ),
            go.Scatter(yaxis="y5" ,x=rdf.index ,y=rdf["80"], name= "80",line={ "color": "black", "width": 0.5 } ),
         
            # 出来高
            go.Bar(yaxis="y6", x=rdf.index, y=rdf["Volume"], name= "Volume", marker={ "color": "slategray" } ) 
        ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

In [ ]:
fig.write_html("{}_{}.html".format(code, name))

In [ ]:
df.to_csv("{}_{}.csv".format(code, name))

In [ ]:
# CSVファイルを読み込んでデータフレームを生成
df = pd.read_csv("{}_{}.csv".format(code, name))
# インデックスを定義
df.index = df["Date"]
df.tail()

,Date,Open,High,Low,Close,Volume,ma5,ma25,gc,dc,...,macd_signal,hist,rsi14,rsi28,70,30,slowK,slowD,80,20
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-01,2022-04-01,2354.0,2372.0,2343.0,2366.5,1931000,2413.7,2354.12,NaN,NaN,...,22.010487,0.301800,48.971481,50.724200,70,30,26.441650,46.516755,80,20
2022-04-04,2022-04-04,2371.0,2383.0,2367.0,2373.5,1298400,2400.4,2356.32,NaN,NaN,...,21.362786,-2.590802,49.961419,51.205886,70,30,14.350265,28.953110,80,20
2022-04-05,2022-04-05,2396.5,2405.0,2359.0,2372.5,1926700,2384.3,2357.38,NaN,NaN,...,20.231138,-4.526593,49.812750,51.131838,70,30,22.216931,21.002949,80,20
2022-04-06,2022-04-06,2349.5,2352.5,2297.0,2299.5,2210300,2356.6,2355.96,NaN,NaN,...,17.644700,-10.345750,40.369025,46.086507,70,30,16.653595,17.740264,80,20
2022-04-07,2022-04-07,2265.5,2268.0,2213.5,2247.5,3709300,2331.9,2354.56,NaN,2354.56,...,13.412169,-16.930126,35.243411,42.955448,70,30,14.438451,17.769659,80,20
